# Video Identification of Suspicious Behavior: Preparation

This notebook will **prepare** your video data by:
* Processing the images by extracting out individual images (using `open-cv`) and saving them to DBFS / cloud storage
* With the saved images, execute Spark Deep Learning Pipelines `DeepImageFeaturizer` to extract image features and saving them to DBFS / cloud storage in Parquet format
 * Perform this task for both the training and test datasets

The source data used in this notebook can be found at [EC Funded CAVIAR project/IST 2001 37540](http://homepages.inf.ed.ac.uk/rbf/CAVIAR/)

<img src="https://databricks.com/wp-content/uploads/2018/09/mnt_raela_video_splash.png" width=900/>


### Cluster Configuration
* Suggested cluster configuration:
 * Databricks Runtime Version: `Databricks Runtime for ML` (e.g. 4.1 ML, 4.2 ML, etc.)
 * Driver: 64GB RAM Instance (e.g. `Azure: Standard_D16s_v3, AWS: r4.4xlarge`)
 * Workers: 2x 64GB RAM Instance (e.g. `Azure: Standard_D16s_v3, AWS: r4.4xlarge`)
 * Python: `Python 3`
 
### Need to install manually
To install, refer to **Upload a Python PyPI package or Python Egg** [Databricks](https://docs.databricks.com/user-guide/libraries.html#upload-a-python-pypi-package-or-python-egg) | [Azure Databricks](https://docs.azuredatabricks.net/user-guide/libraries.html#upload-a-python-pypi-package-or-python-egg)

* Python Libraries:
 * `opencv-python`: 3.4.2 
 
### Libraries Already Included in Databricks Runtime for ML
Because we're using *Databricks Runtime for ML*, you do **not** need to install the following libraires
* Python Libraries:
 * `h5py`: 2.7.1
 * `tensorflow`: 1.7.1
 * `keras`: 2.1.5 (Using TensorFlow backend)
 * *You can check by `import tensorflow as tf; print(tf.__version__)`*

* JARs:
 * `spark-deep-learning-1.0.0-spark2.3-s_2.11.jar`
 * `tensorframes-0.3.0-s_2.11.jar`
 * *You can check by reviewing cluster's Spark UI > Environment)*

In [2]:
%run ./video_config

In [3]:
display(dbutils.fs.ls(srcVideoPath))

path,name,size
dbfs:/databricks-datasets/cctvVideos/train/Browse2.mpg,Browse2.mpg,10005185
dbfs:/databricks-datasets/cctvVideos/train/Browse3.mpg,Browse3.mpg,10503996
dbfs:/databricks-datasets/cctvVideos/train/Browse4.mpg,Browse4.mpg,13152710
dbfs:/databricks-datasets/cctvVideos/train/Browse_WhileWaiting1.mpg,Browse_WhileWaiting1.mpg,9033442
dbfs:/databricks-datasets/cctvVideos/train/Fight_Chase.mpg,Fight_Chase.mpg,5087190
dbfs:/databricks-datasets/cctvVideos/train/Fight_RunAway1.mpg,Fight_RunAway1.mpg,6500613
dbfs:/databricks-datasets/cctvVideos/train/LeftBag.mpg,LeftBag.mpg,17201025
dbfs:/databricks-datasets/cctvVideos/train/LeftBox.mpg,LeftBox.mpg,10185955
dbfs:/databricks-datasets/cctvVideos/train/Rest_WiggleOnFloor.mpg,Rest_WiggleOnFloor.mpg,15097843
dbfs:/databricks-datasets/cctvVideos/train/Walk1.mpg,Walk1.mpg,7208095


Here is an example video that we will perform our training on to identify suspicious behavior.
* The source of this data is from the [EC Funded CAVIAR project/IST 2001 37540](http://homepages.inf.ed.ac.uk/rbf/CAVIAR/).

![](https://databricks.com/wp-content/uploads/2018/09/Browse2.gif)

Extract JPG images from MPG videos using OpenCV (`cv2`)

In [6]:
# Extract and Save Images using CV2
def extractImagesSave(src, tgt):
  import cv2
  import uuid
  import re

  ## Extract one video frame per second and save frame as JPG
  def extractImages(pathIn):
      count = 0
      srcVideos = "/dbfs" + src + "(.*).mpg"
      p = re.compile(srcVideos)
      vidName = str(p.search(pathIn).group(1))
      vidcap = cv2.VideoCapture(pathIn)
      success,image = vidcap.read()
      success = True
      while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))
        success,image = vidcap.read()
        print ('Read a new frame: ', success)
        cv2.imwrite("/dbfs" + tgt + vidName + "frame%04d.jpg" % count, image)     # save frame as JPEG file
        count = count + 1
        print ('Wrote a new frame')
        
  ## Extract frames from all videos and save in s3 folder
  def createFUSEpaths(dbfsFilePath):
    return "/dbfs/" + dbfsFilePath[0][6:]
  
  # Build up fileList RDD
  fileList = dbutils.fs.ls(src)
  FUSEfileList = map(createFUSEpaths, fileList)
  FUSEfileList_rdd = sc.parallelize(FUSEfileList)
  
  # Ensure directory is created
  dbutils.fs.mkdirs(tgt)
  
  # Extract and save images
  FUSEfileList_rdd.map(extractImages).count()

  
# Remove Empty Files
def removeEmptyFiles(pathDir):
  import sys
  import os

  rootDir = '/dbfs' + pathDir
  for root, dirs, files in os.walk(rootDir):
    for f in files:
      fileName = os.path.join(root, f)
      if os.path.getsize(fileName) == 0:
        print ("empty fileName: %s \n" % fileName)
        os.remove(fileName)


# Training Dataset

In [8]:
# Extract Images
extractImagesSave(srcVideoPath, targetImgPath)

# Remove Empty Files
removeEmptyFiles(targetImgPath)

# View file list of images extracted from video
display(dbutils.fs.ls(targetImgPath))

path,name,size
dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0000.jpg,Browse2frame0000.jpg,45025
dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0001.jpg,Browse2frame0001.jpg,52545
dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0002.jpg,Browse2frame0002.jpg,52801
dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0003.jpg,Browse2frame0003.jpg,53165
dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0004.jpg,Browse2frame0004.jpg,53307
dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0005.jpg,Browse2frame0005.jpg,53787
dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0006.jpg,Browse2frame0006.jpg,45644
dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0007.jpg,Browse2frame0007.jpg,49379
dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0008.jpg,Browse2frame0008.jpg,50835
dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0009.jpg,Browse2frame0009.jpg,51129


In [9]:
from pyspark.ml.image import ImageSchema

trainImages = ImageSchema.readImages(targetImgPath)
display(trainImages)

image
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0000.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0001.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0002.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0003.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0004.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0005.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0006.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0007.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0008.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/train/Browse2frame0009.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"


Use [Spark Deep Learning Pipelines](https://github.com/databricks/spark-deep-learning) `DeepImageFeaturizer` to build image features via the InceptionV3 model

In [11]:
# Save Image Features using 
def saveImageFeatures(images, filePath):
  from sparkdl import DeepImageFeaturizer

  # Build featurizer using DeepImageFeaturizer and the InceptionV3 model
  featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")

  # Transform images to pull out image (origin, height, width, nChannels, mode, data) and features (udt)
  features = featurizer.transform(images)

  # Push feature information into Parquet file format
  # This might take a few minutes
  dbutils.fs.mkdirs(filePath)

  # Extract only image file name (imgFileName) within our saved features
  features.select("image.origin", "features").coalesce(2).write.mode("overwrite").parquet(filePath)

In [12]:
saveImageFeatures(trainImages, imgFeaturesPath)

Using TensorFlow backend.

In [13]:
# View Parquet Features
display(dbutils.fs.ls(imgFeaturesPath))

path,name,size
dbfs:/mnt/tardis6/videos/cctv_features/train/_SUCCESS,_SUCCESS,0
dbfs:/mnt/tardis6/videos/cctv_features/train/_committed_461830657016176776,_committed_461830657016176776,120
dbfs:/mnt/tardis6/videos/cctv_features/train/_started_461830657016176776,_started_461830657016176776,0
dbfs:/mnt/tardis6/videos/cctv_features/train/part-00000-tid-461830657016176776-282d96cc-7c6a-45a8-9c7f-ee0cb09f5f0b-11-c000.snappy.parquet,part-00000-tid-461830657016176776-282d96cc-7c6a-45a8-9c7f-ee0cb09f5f0b-11-c000.snappy.parquet,176071398


# Test Dataset

In [15]:
display(dbutils.fs.ls(srcTestVideoPath))

path,name,size
dbfs:/databricks-datasets/cctvVideos/test/Browse1.mpg,Browse1.mpg,12405350
dbfs:/databricks-datasets/cctvVideos/test/Browse_WhileWaiting2.mpg,Browse_WhileWaiting2.mpg,21395638
dbfs:/databricks-datasets/cctvVideos/test/Fight_OneManDown.mpg,Fight_OneManDown.mpg,11320119
dbfs:/databricks-datasets/cctvVideos/test/Fight_RunAway2.mpg,Fight_RunAway2.mpg,6483410
dbfs:/databricks-datasets/cctvVideos/test/LeftBag_AtChair.mpg,LeftBag_AtChair.mpg,12904113
dbfs:/databricks-datasets/cctvVideos/test/LeftBag_BehindChair.mpg,LeftBag_BehindChair.mpg,12593939
dbfs:/databricks-datasets/cctvVideos/test/LeftBox.mpg,LeftBox.mpg,10185955
dbfs:/databricks-datasets/cctvVideos/test/Rest_SlumpOnFloor.mpg,Rest_SlumpOnFloor.mpg,10634883
dbfs:/databricks-datasets/cctvVideos/test/Walk3.mpg,Walk3.mpg,15889216


In [16]:
# Extract Images
extractImagesSave(srcTestVideoPath, targetImgTestPath)

# Remove Empty Files
removeEmptyFiles(targetImgTestPath)

# View file list of images extracted from video
display(dbutils.fs.ls(targetImgTestPath))

path,name,size
dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0000.jpg,Browse1frame0000.jpg,45922
dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0001.jpg,Browse1frame0001.jpg,54227
dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0002.jpg,Browse1frame0002.jpg,53948
dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0003.jpg,Browse1frame0003.jpg,53802
dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0004.jpg,Browse1frame0004.jpg,53689
dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0005.jpg,Browse1frame0005.jpg,54099
dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0006.jpg,Browse1frame0006.jpg,45806
dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0007.jpg,Browse1frame0007.jpg,49294
dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0008.jpg,Browse1frame0008.jpg,49152
dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0009.jpg,Browse1frame0009.jpg,51930


In [17]:
from pyspark.ml.image import ImageSchema

testImages = ImageSchema.readImages(targetImgTestPath)
display(testImages)

image
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0000.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0001.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0002.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0003.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0004.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0005.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0006.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0007.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0008.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/mnt/tardis6/videos/cctvFrames/test/Browse1frame0009.jpg, 288, 384, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"


In [18]:
saveImageFeatures(testImages, imgFeaturesTestPath)

In [19]:
# View Parquet Features
display(dbutils.fs.ls(imgFeaturesTestPath))

path,name,size
dbfs:/mnt/tardis6/videos/cctv_features/test/_SUCCESS,_SUCCESS,0
dbfs:/mnt/tardis6/videos/cctv_features/test/_committed_4373318494123618804,_committed_4373318494123618804,121
dbfs:/mnt/tardis6/videos/cctv_features/test/_started_4373318494123618804,_started_4373318494123618804,0
dbfs:/mnt/tardis6/videos/cctv_features/test/part-00000-tid-4373318494123618804-4e56fe1c-a029-4730-b945-25453cfc1b95-21-c000.snappy.parquet,part-00000-tid-4373318494123618804-4e56fe1c-a029-4730-b945-25453cfc1b95-21-c000.snappy.parquet,172615941
